In [1]:
import scipy.io as sio
import numpy as np
import os
import gc
import six.moves.urllib as urllib
import cv2
import time
import xml.etree.cElementTree as ET
import random
import shutil as sh
from shutil import copyfile
import zipfile

import csv


def save_csv(csv_path, csv_content):
    with open(csv_path, 'w') as csvfile:
        wr = csv.writer(csvfile)
        for i in range(len(csv_content)):
            wr.writerow(csv_content[i])


def get_bbox_visualize(base_path, dir):
    image_path_array = []
    for root, dirs, filenames in os.walk(base_path + dir):
        for f in filenames:
            if(f.split(".")[1] == "jpg"):
                img_path = base_path + dir + "/" + f
                image_path_array.append(img_path)

    #sort image_path_array to ensure its in the low to high order expected in polygon.mat
    image_path_array.sort()
    boxes = sio.loadmat(
        base_path + dir + "/polygons.mat")
    # there are 100 of these per folder in the egohands dataset
    polygons = boxes["polygons"][0]
    #first = polygons[0]
    # print(len(first))
    pointindex = 0

    for first in polygons:
        index = 0

        font = cv2.FONT_HERSHEY_SIMPLEX

        img_id = image_path_array[pointindex]
        img = cv2.imread(img_id)

        img_params = {}
        img_params["width"] = np.size(img, 1)
        img_params["height"] = np.size(img, 0)
        head, tail = os.path.split(img_id)
        img_params["filename"] = tail
        img_params["path"] = os.path.abspath(img_id)
        img_params["type"] = "train"
        pointindex += 1

        boxarray = []
        csvholder = []
        for pointlist in first:
            pst = np.empty((0, 2), int)
            max_x = max_y = min_x = min_y = height = width = 0

            findex = 0
            for point in pointlist:
                if(len(point) == 2):
                    x = int(point[0])
                    y = int(point[1])

                    if(findex == 0):
                        min_x = x
                        min_y = y
                    findex += 1
                    max_x = x if (x > max_x) else max_x
                    min_x = x if (x < min_x) else min_x
                    max_y = y if (y > max_y) else max_y
                    min_y = y if (y < min_y) else min_y
                    # print(index, "====", len(point))
                    appeno = np.array([[x, y]])
                    pst = np.append(pst, appeno, axis=0)
                    cv2.putText(img, ".", (x, y), font, 0.7,
                                (255, 255, 255), 2, cv2.LINE_AA)

            hold = {}
            hold['minx'] = min_x
            hold['miny'] = min_y
            hold['maxx'] = max_x
            hold['maxy'] = max_y
            if (min_x > 0 and min_y > 0 and max_x > 0 and max_y > 0):
                boxarray.append(hold)
                labelrow = [tail,
                            np.size(img, 1), np.size(img, 0), "hand", min_x, min_y, max_x, max_y]
                csvholder.append(labelrow)

            cv2.polylines(img, [pst], True, (0, 255, 255), 1)
            cv2.rectangle(img, (min_x, max_y),
                          (max_x, min_y), (0, 255, 0), 1)

        csv_path = img_id.split(".")[0]
        if not os.path.exists(csv_path + ".csv"):
            cv2.putText(img, "DIR : " + dir + " - " + tail, (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (77, 255, 9), 2)
            #cv2.imshow('Verifying annotation ', img)
            save_csv(csv_path + ".csv", csvholder)
            print("===== saving csv file for ", tail)
        cv2.waitKey(2)  # close window when a key press is detected


def create_directory(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# combine all individual csv files for each image into a single csv file per folder.


def generate_label_files(image_dir):
    header = ['filename', 'width', 'height',
              'class', 'xmin', 'ymin', 'xmax', 'ymax']
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            csvholder = []
            csvholder.append(header)
            loop_index = 0
            for f in os.listdir(image_dir + dir):
                if(f.split(".")[1] == "csv"):
                    loop_index += 1
                    #print(loop_index, f)
                    csv_file = open(image_dir + dir + "/" + f, 'r')
                    reader = csv.reader(csv_file)
                    for row in reader:
                        csvholder.append(row)
                    csv_file.close()
                    os.remove(image_dir + dir + "/" + f)
            save_csv(image_dir + dir + "/" + dir + "_labels.csv", csvholder)
            print("Saved label csv for ", dir, image_dir +
                  dir + "/" + dir + "_labels.csv")


# Split data, copy to train/test folders
def split_data_test_eval_train(image_dir):
    create_directory("/work/handtracking-master/egohands/images")
    create_directory("/work/handtracking-master/egohands/images/train")
    create_directory("/work/handtracking-master/egohands/images/test")

    data_size = 4000
    loop_index = 0
    data_sampsize = int(0.1 * data_size)
    test_samp_array = random.sample(range(data_size), k=data_sampsize)

    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            for f in os.listdir(image_dir + dir):
                if(f.split(".")[1] == "jpg"):
                    loop_index += 1
                    print(loop_index, f)

                    if loop_index in test_samp_array:
                        os.rename(image_dir + dir +
                                  "/" + f, "/work/handtracking-master/egohands/images/test/" + f)
                        os.rename(image_dir + dir +
                                  "/" + f.split(".")[0] + ".csv", "/work/handtracking-master/egohands/images/test/" + f.split(".")[0] + ".csv")
                    else:
                        os.rename(image_dir + dir +
                                  "/" + f, "/work/handtracking-master/egohands/images/train/" + f)
                        os.rename(image_dir + dir +
                                  "/" + f.split(".")[0] + ".csv", "/work/handtracking-master/egohands/images/train/" + f.split(".")[0] + ".csv")
                    print(loop_index, image_dir + f)
            print(">   done scanning director ", dir)
            os.remove(image_dir + dir + "/polygons.mat")
            os.rmdir(image_dir + dir)

        print("Train/test content generation complete!")
        generate_label_files("/work/handtracking-master/egohands/images/")


def generate_csv_files(image_dir):
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            get_bbox_visualize(image_dir, dir)

    print("CSV generation complete!\nGenerating train/test/eval folders")
    #split_data_test_eval_train("/work/handtracking-master/egohands/_LABELLED_SAMPLES/")


# rename image files so we can have them all in a train/test/eval folder.
def rename_files(image_dir):
    print("Renaming files")
    loop_index = 0
    for root, dirs, filenames in os.walk(image_dir):
        for dir in dirs:
            for f in os.listdir(image_dir + dir):
                if (dir not in f):
                    if(f.split(".")[1] == "jpg"):
                        loop_index += 1
                        os.rename(image_dir + dir +
                                  "/" + f, image_dir + dir +
                                  "/" + dir + "_" + f)
                else:
                    break

    #generate_csv_files("/work/handtracking-master/egohands/_LABELLED_SAMPLES/")

def extract_folder(dataset_path):
    print("Egohands dataset already downloaded.\nGenerating CSV files")
    if not os.path.exists("egohands"):
        zip_ref = zipfile.ZipFile(dataset_path, 'r')
        print("> Extracting Dataset files")
        zip_ref.extractall("egohands")
        print("> Extraction complete")
        zip_ref.close()
        rename_files("/work/handtracking-master/egohands/_LABELLED_SAMPLES/")

def download_egohands_dataset(dataset_url, dataset_path):
    is_downloaded = os.path.exists(dataset_path)
    if not is_downloaded:
        print(
            "> downloading egohands dataset. This may take a while (1.3GB, say 3-5mins). Coffee break?")
        opener = urllib.request.URLopener()
        opener.retrieve(dataset_url, dataset_path)
        print("> download complete")
        extract_folder(dataset_path);

    else:
        extract_folder(dataset_path)


EGOHANDS_DATASET_URL = "http://vision.soic.indiana.edu/egohands_files/egohands_data.zip"
EGO_HANDS_FILE = "/work/handtracking-master/egohands_data.zip"


#download_egohands_dataset(EGOHANDS_DATASET_URL, EGO_HANDS_FILE)

In [ ]:
#extract_folder("/work/handtracking-master/egohands_data.zip")

In [3]:
rename_files("/work/handtracking-master/egohands/_LABELLED_SAMPLES/")

Renaming files


In [4]:
generate_csv_files("/work/handtracking-master/egohands/_LABELLED_SAMPLES/")

===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0017.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0018.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0058.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0106.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0138.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0181.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0185.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0264.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0294.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0323.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0361.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0363.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0443.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0487.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0551.jpg
===== saving csv file for  PUZZLE_COURTYARD_B_S_frame_0

===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0779.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0793.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0811.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0822.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0859.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0909.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0911.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0938.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_0943.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1009.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1015.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1066.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1095.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1122.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1159.jpg
===== saving csv file for  PUZZLE_OFFICE_B_H_frame_1210.jpg
===== saving csv file for  PUZZLE_OFFICE

===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1827.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1853.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1889.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1918.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1942.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1957.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1959.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_1962.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2015.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2031.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2130.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2132.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2188.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2231.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2234.jpg
===== saving csv file for  CHESS_LIVINGROOM_S_B_frame_2

===== saving csv file for  CHESS_OFFICE_B_S_frame_0085.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0093.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0096.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0124.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0145.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0204.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0262.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0263.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0321.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0342.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0350.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0374.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0383.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0403.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0425.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0437.jpg
===== saving csv file for  CHESS_OFFICE_B_S_frame_0439.j

===== saving csv file for  JENGA_COURTYARD_B_H_frame_0900.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_0921.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_0934.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_0994.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1019.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1068.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1077.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1086.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1097.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1123.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1162.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1195.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1206.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1291.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1292.jpg
===== saving csv file for  JENGA_COURTYARD_B_H_frame_1294.jpg
===== sa

...

===== saving csv file for  CHESS_OFFICE_H_T_frame_0099.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0121.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0132.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0133.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0142.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0199.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0207.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0283.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0299.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0325.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0348.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0445.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0481.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0492.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0548.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0568.jpg
===== saving csv file for  CHESS_OFFICE_H_T_frame_0594.j

===== saving csv file for  CHESS_COURTYARD_H_S_frame_1091.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1145.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1146.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1148.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1161.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1167.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1187.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1213.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1254.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1256.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1297.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1298.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1353.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1363.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1396.jpg
===== saving csv file for  CHESS_COURTYARD_H_S_frame_1427.jpg
===== sa

===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1763.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1793.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1894.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1916.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1938.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1953.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_1968.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2022.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2043.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2085.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2119.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2172.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2267.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2273.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2277.jpg
===== saving csv file for  PUZZLE_COURTYARD_T_H_frame_2

===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0300.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0317.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0368.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0453.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0460.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0493.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0499.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0525.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0534.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0560.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0598.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0609.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0614.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0622.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0625.jpg
===== saving csv file for  CARDS_LIVINGROOM_H_S_frame_0

===== saving csv file for  JENGA_COURTYARD_S_T_frame_1194.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1209.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1216.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1249.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1266.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1270.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1338.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1390.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1409.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1410.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1434.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1437.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1471.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1497.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1506.jpg
===== saving csv file for  JENGA_COURTYARD_S_T_frame_1545.jpg
===== sa

In [5]:
split_data_test_eval_train("/work/handtracking-master/egohands/_LABELLED_SAMPLES/")

1 PUZZLE_COURTYARD_B_S_frame_2176.jpg
1 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_2176.jpg
2 PUZZLE_COURTYARD_B_S_frame_2159.jpg
2 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_2159.jpg
3 PUZZLE_COURTYARD_B_S_frame_0866.jpg
3 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_0866.jpg
4 PUZZLE_COURTYARD_B_S_frame_0718.jpg
4 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_0718.jpg
5 PUZZLE_COURTYARD_B_S_frame_2127.jpg
5 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_2127.jpg
6 PUZZLE_COURTYARD_B_S_frame_0757.jpg
6 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_0757.jpg
7 PUZZLE_COURTYARD_B_S_frame_0264.jpg
7 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_B_S_frame_0264.jpg
8 PUZZLE_COURTYARD_B_S_frame_1779.jpg
8 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZ

141 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_2102.jpg
142 PUZZLE_OFFICE_B_H_frame_1371.jpg
142 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_1371.jpg
143 PUZZLE_OFFICE_B_H_frame_0015.jpg
143 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0015.jpg
144 PUZZLE_OFFICE_B_H_frame_0911.jpg
144 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0911.jpg
145 PUZZLE_OFFICE_B_H_frame_2159.jpg
145 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_2159.jpg
146 PUZZLE_OFFICE_B_H_frame_0771.jpg
146 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0771.jpg
147 PUZZLE_OFFICE_B_H_frame_1095.jpg
147 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_1095.jpg
148 PUZZLE_OFFICE_B_H_frame_0811.jpg
148 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_B_H_frame_0811.jpg
149 PUZZLE_OFFICE_B_H

214 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_1339.jpg
215 CHESS_LIVINGROOM_S_B_frame_1572.jpg
215 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_1572.jpg
216 CHESS_LIVINGROOM_S_B_frame_0466.jpg
216 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0466.jpg
217 CHESS_LIVINGROOM_S_B_frame_0620.jpg
217 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0620.jpg
218 CHESS_LIVINGROOM_S_B_frame_0922.jpg
218 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0922.jpg
219 CHESS_LIVINGROOM_S_B_frame_2459.jpg
219 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_2459.jpg
220 CHESS_LIVINGROOM_S_B_frame_0752.jpg
220 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0752.jpg
221 CHESS_LIVINGROOM_S_B_frame_0639.jpg
221 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVING

276 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_2188.jpg
277 CHESS_LIVINGROOM_S_B_frame_0163.jpg
277 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0163.jpg
278 CHESS_LIVINGROOM_S_B_frame_0242.jpg
278 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0242.jpg
279 CHESS_LIVINGROOM_S_B_frame_0029.jpg
279 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0029.jpg
280 CHESS_LIVINGROOM_S_B_frame_1674.jpg
280 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_1674.jpg
281 CHESS_LIVINGROOM_S_B_frame_2584.jpg
281 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_2584.jpg
282 CHESS_LIVINGROOM_S_B_frame_0689.jpg
282 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B_frame_0689.jpg
283 CHESS_LIVINGROOM_S_B_frame_1072.jpg
283 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CHESS_LIVING

340 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_2118.jpg
341 PUZZLE_COURTYARD_H_T_frame_1472.jpg
341 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_1472.jpg
342 PUZZLE_COURTYARD_H_T_frame_1832.jpg
342 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_1832.jpg
343 PUZZLE_COURTYARD_H_T_frame_2432.jpg
343 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_2432.jpg
344 PUZZLE_COURTYARD_H_T_frame_0373.jpg
344 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_0373.jpg
345 PUZZLE_COURTYARD_H_T_frame_1519.jpg
345 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_1519.jpg
346 PUZZLE_COURTYARD_H_T_frame_0957.jpg
346 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T_frame_0957.jpg
347 PUZZLE_COURTYARD_H_T_frame_0343.jpg
347 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURT

...

4426 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_1916.jpg
4427 PUZZLE_COURTYARD_T_H_frame_0553.jpg
4427 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_0553.jpg
4428 PUZZLE_COURTYARD_T_H_frame_2482.jpg
4428 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_2482.jpg
4429 PUZZLE_COURTYARD_T_H_frame_1253.jpg
4429 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_1253.jpg
4430 PUZZLE_COURTYARD_T_H_frame_2172.jpg
4430 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_2172.jpg
4431 PUZZLE_COURTYARD_T_H_frame_1602.jpg
4431 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_1602.jpg
4432 PUZZLE_COURTYARD_T_H_frame_1600.jpg
4432 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H_frame_1600.jpg
4433 PUZZLE_COURTYARD_T_H_frame_0049.jpg
4433 /work/handtracking-master/egohands/_LABELLED_SAMPL

4538 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1985.jpg
4539 PUZZLE_OFFICE_T_S_frame_1014.jpg
4539 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1014.jpg
4540 PUZZLE_OFFICE_T_S_frame_2619.jpg
4540 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_2619.jpg
4541 PUZZLE_OFFICE_T_S_frame_1821.jpg
4541 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1821.jpg
4542 PUZZLE_OFFICE_T_S_frame_0137.jpg
4542 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_0137.jpg
4543 PUZZLE_OFFICE_T_S_frame_2447.jpg
4543 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_2447.jpg
4544 PUZZLE_OFFICE_T_S_frame_1535.jpg
4544 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1535.jpg
4545 PUZZLE_OFFICE_T_S_frame_1555.jpg
4545 /work/handtracking-master/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S_frame_1555.jpg
4546 P

4633 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_0905.jpg
4634 CARDS_LIVINGROOM_H_S_frame_1476.jpg
4634 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_1476.jpg
4635 CARDS_LIVINGROOM_H_S_frame_1924.jpg
4635 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_1924.jpg
4636 CARDS_LIVINGROOM_H_S_frame_1145.jpg
4636 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_1145.jpg
4637 CARDS_LIVINGROOM_H_S_frame_0598.jpg
4637 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_0598.jpg
4638 CARDS_LIVINGROOM_H_S_frame_0453.jpg
4638 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_0453.jpg
4639 CARDS_LIVINGROOM_H_S_frame_2174.jpg
4639 /work/handtracking-master/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S_frame_2174.jpg
4640 CARDS_LIVINGROOM_H_S_frame_2103.jpg
4640 /work/handtracking-master/egohands/_LABELLED_SAMPL

4738 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_0315.jpg
4739 JENGA_COURTYARD_S_T_frame_1644.jpg
4739 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_1644.jpg
4740 JENGA_COURTYARD_S_T_frame_1110.jpg
4740 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_1110.jpg
4741 JENGA_COURTYARD_S_T_frame_1780.jpg
4741 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_1780.jpg
4742 JENGA_COURTYARD_S_T_frame_0940.jpg
4742 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_0940.jpg
4743 JENGA_COURTYARD_S_T_frame_0235.jpg
4743 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_0235.jpg
4744 JENGA_COURTYARD_S_T_frame_0353.jpg
4744 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T_frame_0353.jpg
4745 JENGA_COURTYARD_S_T_frame_2617.jpg
4745 /work/handtracking-master/egohands/_LABELLED_SAMPLES/JENGA_COURT

Saved label csv for  train /work/handtracking-master/egohands/images/train/train_labels.csv
Saved label csv for  test /work/handtracking-master/egohands/images/test/test_labels.csv
